In [29]:
# default_exp callbacks

In [30]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
from fastcore.test import *

In [31]:
#export
from contextlib import ContextDecorator
from rich.console import Console
from fastcore.imports import *
from fastcore.foundation import *
from fastcore.basics import *
from fastcore.meta import *

In [32]:
#export
_events = L.split('after_enter before_exit')

In [33]:
#export
mk_class('event', **_events.map_dict(),
         doc="All possible events as attributes to get tab-completion and typo-proofing")

In [34]:
#export
@funcs_kwargs(as_method=True)
class Callback(Stateful,GetAttr):
    "Basic class handling tweaks of the runner"
    _methods = _events

    def __init__(self, **kwargs): assert not kwargs, f'Passed unknown events: {kwargs}'
    def __repr__(self): return type(self).__name__

    def __call__(self, event_name):
        "Call `self.{event_name}` if it's defined"
        res = getattr(self, event_name, noop)()
        return res

    def __setattr__(self, name, value):
        super().__setattr__(name, value)

    @property
    def name(self):
        "Name of the `Callback`, camel-cased and with '*Callback*' removed"
        return class2attr(self, 'Callback')

The `Callback` can be used to define callbacks to intorduce new tweaks into the test runner. Currently there are two life cycle events of the test runner.

* `after_enter` - called after entering into Describe or It.
* `before_exit` - called before exiting the Describe or It.

> The `Callback` class is hugely inspired by the beautiful FastAI library. Infact the code for the callback is the direct copy of the Callback class in the FastAI class. Some of the tests are also from the FastAI library.

In [35]:
show_doc(Callback.__call__)

<h4 id="Callback.__call__" class="doc_header"><code>Callback.__call__</code><a href="__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>Callback.__call__</code>(**`event_name`**)

Call `self.{event_name}` if it's defined

In [36]:
show_doc(Callback.name)

<h4 id="Callback.name" class="doc_header"><code>Callback.name</code><a href="" class="source_link" style="float:right">[source]</a></h4>

Name of the [`Callback`](/mochaccino/suite.html#Callback), camel-cased and with '*Callback*' removed

A callback can be created by subclassing the `Callback` class like this👇

In [37]:
class _T(Callback):
    def call_me(self): return "maybe"
test_eq(_T()("call_me"), "maybe")

You can also create a function and then assign it to an event in the Callback class. Doing this would give you a callback for that event. 

👀 at the test below for an example about how to assign a function as a callback event.

In [38]:
def cb(self): return "maybe"
_t = Callback(after_enter=cb)
test_eq(_t(event.after_enter), "maybe")

In [39]:
#export
console = Console()

In [40]:
#export
class CallbackHandler:
    "Basic handler to assign the callback events to their respective flows"
    def __init__(self,cbs=None): self.cbs = cbs if cbs else []

    def after_enter(self, o):
        res = True
        for cb in self.cbs: res = res and cb.after_enter(o)
        return res

    def before_exit(self, o):
        res = True
        for cb in self.cbs: res = res and cb.before_exit(o)
        return res

The `CallbackHandler` helps in collecting the callbacks that are passed and then assigns each callback to it'sown type.

In [41]:
#export
class ReporterCallback(Callback):
    def after_enter(self): console.print(self.desc, style="green")
    def before_exit(self): console.print(self.desc, style="green")

In [42]:
notebook2script()

Converted 00_callbacks.ipynb.
Converted 01_suite.ipynb.
